In [36]:
 pip install --upgrade pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 29.7 MB/s eta 0:00:000m eta 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
Note: you may need to restart the kernel to use updated packages.


In [37]:
!pip install PyMuPDF


In [1]:
import io
import fitz  # PyMuPDF
import json

In [2]:
pdf_path='/workspaces/ollama/files/ring.pdf'

In [3]:
# Open the PDF file
pdf_document = fitz.open(pdf_path)

In [4]:
print(pdf_document)

Document('/workspaces/ollama/files/ring.pdf')


In [5]:
text = []
# Iterate through each page
for page_num in range(pdf_document.page_count):
    # Get the page
    page = pdf_document.load_page(page_num)
    
    # Extract text from the page
    text.append(page.get_text())
   
    # Print the text content of the page
   # print(f"Page {page_num + 1}:\n{text}\n")


In [6]:
result = ''.join(text)
#print(result)

In [7]:
byte_data = result.encode('utf-8')

In [8]:
# Example: Read PDF data from a file and store it in byte_data
with open(pdf_path, 'rb') as file:
    byte_data = file.read()  # Read the PDF data into bytes

with io.BytesIO(byte_data) as f_in:
    doc = fitz.open("pdf", f_in.read())  # Open directly from bytes

In [9]:
def is_title(text, font_size, threshold_size=10):
    return font_size >= threshold_size  # Adjust the threshold as needed

def extract_titles_from_pdf(pdf_path):
    document = fitz.open(pdf_path)
    titles = []

    for page in document:
        text_blocks = page.get_text("dict")["blocks"]
        for block in text_blocks:
            if "lines" in block:  # Only consider text blocks
                for line in block["lines"]:
                    for span in line["spans"]:
                        text = span["text"].strip()
                        font_size = span["size"]
                        if is_title(text, font_size):
                            #titles.append({text:''.join(page.get_text())})
                             titles.append({"title": text, "description": ''.join(page.get_text())})

    document.close()
    return titles

pdf_path = pdf_path  # Replace with your PDF file path
titles = extract_titles_from_pdf(pdf_path)
print("Detected Titles:", titles)

Detected Titles: [{'title': 'E-SCOOTER', 'description': 'E-SCOOTER\nUSER MANUAL\nORIGINAL INSTRUCTIONS\n'}, {'title': 'USER MANUAL', 'description': 'E-SCOOTER\nUSER MANUAL\nORIGINAL INSTRUCTIONS\n'}, {'title': 'Contents', 'description': 'Contents\n1\n2\n3\n4\n5\n6\n7\n8\n9\n10\n11\n12\n13\n14\n15\n16\n17\nSafety Instructions \nPacking List \nProduct Function Diagram\nControl Panel/Dashboard\nAssembling Your KickScooter\nFirst Ride\nCharging\nLearning to Ride \nSecurity Tips\nFolding and Carrying\nSpeciﬁcations\nMaintenance\nBefore Your Ride\nSafety Alerts & Error Messages \nTrademarks and Legal Notices\nCertiﬁcations\nContact\n01\n04\n05\n06\n07\n08\n09\n10\n11\n13\n14\n16\n17\n18\n19\n19\n20\n'}, {'title': '', 'description': 'Contents\n1\n2\n3\n4\n5\n6\n7\n8\n9\n10\n11\n12\n13\n14\n15\n16\n17\nSafety Instructions \nPacking List \nProduct Function Diagram\nControl Panel/Dashboard\nAssembling Your KickScooter\nFirst Ride\nCharging\nLearning to Ride \nSecurity Tips\nFolding and Carrying\

In [10]:
with open('ringmanual.json', 'wt') as f_out:
    json.dump(titles, f_out, indent=2)

In [11]:
from openai import OpenAI

In [12]:
from openai import OpenAI

client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)

In [13]:
from elasticsearch import Elasticsearch

In [14]:
es_client = Elasticsearch('http://localhost:9200') 

In [15]:
es_client.info()

ObjectApiResponse({'name': 'a1b506278180', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'Z_IPrBQ8R7a1tEpsWIT6VA', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [16]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "description": {"type": "text"},
            "title": {"type": "keyword"} 
        }
    }
}

index_name = "usermanual-answerrr" #database name maybe ?

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'usermanual-answerrr'})

In [17]:
from tqdm.auto import tqdm

In [18]:

for title in tqdm(titles):
    es_client.index(index=index_name,document=title) # final init of elastiserach

  0%|          | 0/21 [00:00<?, ?it/s]

In [28]:
query = 'Do I need to wear helmet while driving scooter  and can you tell me can I get hurt if I dont wear helmet?'
#query = 'Do I need to wear helmet while driving scooter ?'

In [29]:
def elastic_search(query):
    serach_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["description^3"],
                        "type": "best_fields"
                    }
                }
               
            }
        }
    }
    response = es_client.search(index=index_name,body=serach_query)
    
    response['hits']['hits']
    
    result_doc = []
    
    for hit in response['hits']['hits']:  
        result_doc.append(hit['_source'])
    return result_doc

In [30]:
elastic_search(query)

[{'title': '1. Safety Instructions',
  'description': 'Electric scooter is a recreational device rather than a transporter. But once you ride it into a public community, it will also act as a transporter. Like \nany riding products, an electric scooter can and is intended to move and it is therefore, of course, possible to get into dangerous situations.\nPlease understand that you can reduce the risks by following all the instructions and warnings in this manual, but you cannot eliminate all the risks. \nRemember that whenever you ride the scooter you risk injury or death from loss of control, collisions, and falls. The electric scooter is intended for \nuse on ﬂat pavement or level ground without bumps and obstacles. Be alert and slow down when entering unfamiliar areas.\nAlways wear a helmet when riding. Use an approved bicycle or scooter helmet that ﬁts properly with the chin strap in place, and provides protection \nfor the back of your head. A long-sleeved shirt, long pants, and a

In [31]:
def build_prompt(query,search_results):
    prompt_template =""" 
You are user manuel interpreter. Answer the QUESTION base on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.If CONTEXT doesn't contain  the answer output NONE


QUESTION: {question}

CONTEXT: {context}
""".strip()

    context  = ""
    for doc in search_results:
        context = context + f"title: {doc['title']}\ndescirption: {doc['description']}\n\n"
    
    prompt = prompt_template.format(question=query,context= context).strip()
    return prompt

def llm(prompt):
    response = client.chat.completions.create(
        model='gemma2:2b',
        #model='phi3',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [32]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [34]:
print(rag(query))

Okay, I understand. You're asking me to combine safety information (from a manual) with product and service details to form a comprehensive guide for customers using INMOTION scooters. Let's break it down into sections and incorporate best practices:

**Part 1: Getting Started** -  Focusing on the scooter itself

* **Title:** "Get Ready to Ride!"
* **Description:** A brief overview of the key safety topics that will be covered in this manual. 
    *"To ensure a safe and enjoyable ride, please read all instructions carefully before using your INMOTION e-Scooter."

**Part 2: Battery Care & Handling** - How to get the most out of your battery safely.
* **Title:** "Power Up & Ride Out!
* **Description:** This section will delve into key points about handling and caring for the battery on a practical level. 

    * **1. Do not touch battery contacts: **  Explain why this is crucial for safety.   
        * How to do it safely:  Point out the need for gloves or appropriate electrical protect